# Getting Started with Relay

**Welcome to Relay** - the agent governance system that creates an "air gap" between autonomous agents and critical actions.

## What You'll Learn

- Understanding manifests and policy evaluation
- Testing policies with different scenarios
- Building a policy-protected payment processor
- Tracing the complete Relay flow

## Prerequisites

Make sure Relay infrastructure is running:
```bash
cd ~/relay/infra
docker-compose up -d
```

## 1. Setup and Imports

In [25]:
import sys
sys.path.insert(0, '../')

import requests
from datetime import datetime, UTC
from uuid import uuid4
import json

# Check if infrastructure is running
try:
    response = requests.get('http://localhost:8181/health', timeout=2)
    print('✅ Relay infrastructure is running')
except:
    print('❌ Relay not running. Start with: cd infra && docker-compose up -d')
    raise

✅ Relay infrastructure is running


## 2. Understanding Manifests

A **Manifest** is the core primitive of Relay. It describes an agent action request with full context:

- **agent**: Who is making the request
- **action**: What they want to do
- **justification**: Why they want to do it
- **environment**: Where it will execute

In [26]:
manifest = {
    "manifest_id": str(uuid4()),
    "timestamp": datetime.now(UTC).isoformat(),
    "agent": {
        "agent_id": "getting-started-agent",
        "org_id": "tutorial-org",
        "user_id": "student@example.com"
    },
    "action": {
        "provider": "stripe",
        "method": "create_payment",
        "parameters": {
            "amount": 3500,  # $35.00 in cents
            "currency": "USD",
            "customer_id": "cus_tutorial_001"
        }
    },
    "justification": {
        "reasoning": "Tutorial payment for getting started guide",
        "confidence_score": 1.0
    },
    "environment": "production"
}

print(json.dumps(manifest, indent=2))

{
  "manifest_id": "da7903f7-d767-470e-af40-4d07b6fb99f4",
  "timestamp": "2026-01-18T07:21:43.844637+00:00",
  "agent": {
    "agent_id": "getting-started-agent",
    "org_id": "tutorial-org",
    "user_id": "student@example.com"
  },
  "action": {
    "provider": "stripe",
    "method": "create_payment",
    "parameters": {
      "amount": 3500,
      "currency": "USD",
      "customer_id": "cus_tutorial_001"
    }
  },
  "justification": {
    "reasoning": "Tutorial payment for getting started guide",
    "confidence_score": 1.0
  },
  "environment": "production"
}


## 3. Policy Evaluation

Relay uses Open Policy Agent (OPA) to evaluate manifests against policies.

In [27]:
response = requests.post(
    'http://localhost:8181/v1/data/relay/policies/main',
    json={'input': manifest}
)

result = response.json()['result']
print(f"✅ Approved: {result.get('allow', False)}")
print(f"Reason: {result.get('reason', 'N/A')}")

✅ Approved: True
Reason: No policy matched


## 4. Testing Different Amounts

Let's test the $50 spending limit policy with various amounts.

In [28]:
def test_payment(amount, description):
    """Test a payment amount against policies."""
    test_manifest = {
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": amount}
        }
    }
    
    response = requests.post(
        'http://localhost:8181/v1/data/relay/policies/main',
        json={'input': test_manifest}
    )
    
    result = response.json()['result']
    approved = result.get('allow', False)
    status = "✅" if approved else "❌"
    
    print(f"{description:30} ${amount/100:6.2f}  {status}")
    if not approved:
        print(f"{'':30}         → {result.get('reason', 'N/A')}")

print("Testing Payment Amounts:")
print("=" * 70)
test_payment(1000, "Small payment")
test_payment(2500, "Medium payment")
test_payment(4999, "Just under limit")
test_payment(5000, "Exactly at limit")
test_payment(5001, "Just over limit")
test_payment(10000, "Large payment")

Testing Payment Amounts:
Small payment                  $ 10.00  ✅
Medium payment                 $ 25.00  ✅
Just under limit               $ 49.99  ✅
Exactly at limit               $ 50.00  ✅
Just over limit                $ 50.01  ❌
                                       → Payment amount exceeds $50.00 limit
Large payment                  $100.00  ❌
                                       → Payment amount exceeds $50.00 limit


## 5. Real-World Example: Payment Processor

Let's build a simple payment processor that uses Relay for governance.

In [29]:
class PaymentProcessor:
    """Payment processor with Relay governance."""
    
    def __init__(self, agent_id="payment-processor-001"):
        self.agent_id = agent_id
        self.transactions = []
    
    def process_payment(self, amount, customer_id, description):
        """Process a payment with policy validation."""
        manifest = {
            "agent": {
                "agent_id": self.agent_id,
                "org_id": "tutorial-org"
            },
            "action": {
                "provider": "stripe",
                "method": "create_payment",
                "parameters": {
                    "amount": amount,
                    "currency": "USD",
                    "customer_id": customer_id
                }
            },
            "justification": {"reasoning": description}
        }
        
        response = requests.post(
            'http://localhost:8181/v1/data/relay/policies/main',
            json={'input': manifest}
        )
        
        result = response.json()['result']
        approved = result.get('allow', False)
        
        if approved:
            transaction = {
                'id': f'ch_{len(self.transactions) + 1}',
                'amount': amount,
                'customer': customer_id,
                'status': 'succeeded'
            }
            self.transactions.append(transaction)
            print(f"✅ ${amount/100:.2f} for {customer_id}")
            return transaction
        else:
            reason = result.get('reason', 'Policy violation')
            print(f"❌ {reason}")
            raise Exception(f"Payment denied: {reason}")
    
    def get_total(self):
        """Get total processed amount."""
        return sum(t['amount'] for t in self.transactions) / 100

# Test the processor
processor = PaymentProcessor()

try:
    processor.process_payment(2500, "cus_001", "Coffee subscription")
    processor.process_payment(4800, "cus_002", "Premium plan upgrade")
    processor.process_payment(6000, "cus_003", "Annual subscription")  # Will fail
except Exception:
    pass

print(f"\nTotal processed: ${processor.get_total():.2f}")
print(f"Transactions: {len(processor.transactions)}")

✅ $25.00 for cus_001
✅ $48.00 for cus_002
❌ Payment amount exceeds $50.00 limit

Total processed: $73.00
Transactions: 2


## 6. The Relay Flow

Here's how Relay works end-to-end:

```
1. Agent prepares action → 2. Build manifest → 3. Policy evaluation
                                                        ↓
4. Audit log ← 5. Execute (if approved) ← Decision (approve/deny)
```

In [30]:
def trace_flow(amount, description):
    """Trace the Relay flow for a payment."""
    manifest = {
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {"amount": amount}
        }
    }
    
    response = requests.post(
        'http://localhost:8181/v1/data/relay/policies/main',
        json={'input': manifest}
    )
    
    result = response.json()['result']
    approved = result.get('allow', False)
    
    print(f"\n{description}: ${amount/100:.2f}")
    if approved:
        print(f"✅ APPROVED → Payment executed and logged")
    else:
        print(f"❌ DENIED → {result.get('reason')}")
        print(f"   Denial logged in audit trail")

print("=" * 70)
trace_flow(3000, "Approved payment")
trace_flow(7000, "Denied payment")
print("=" * 70)


Approved payment: $30.00
✅ APPROVED → Payment executed and logged

Denied payment: $70.00
❌ DENIED → Payment amount exceeds $50.00 limit
   Denial logged in audit trail


## Key Takeaways

- **Manifests** describe agent actions with full context
- **Policies** determine what's allowed or denied
- **Deterministic** decisions - same input always gives same output
- **Audit trail** - every decision is logged for forensics
- **Fast** - policy evaluation in milliseconds

## Next Steps

1. **[02_adversarial_prompt_protection.ipynb](02_adversarial_prompt_protection.ipynb)** - Learn about security threats
2. **[03_langchain_integration.ipynb](03_langchain_integration.ipynb)** - Integrate Relay with LangChain
3. **[04_company_policies.ipynb](04_company_policies.ipynb)** - Write custom policies
4. **[05_real_world_scenarios.ipynb](05_real_world_scenarios.ipynb)** - See the business case